# Katib 실습
katib을 사용하여 training code `01-covid19-katib-train.py`를 hyper parameter tunning한다

# 사전준비: Base Image
Host에 ssh 접속하여 아래와 같이 docker build하고 push한다

```sh
cat << EOF > Dockerfile.kf-base
FROM ubuntu:18.04
ARG DEBIAN_FRONTEND=noninteractive

RUN apt-get update && \
    apt-get install -y python3 python3-pip python3-dev ffmpeg libsm6 libxext6 

RUN  pip3 install --upgrade pip && \
     pip install \
        tensorflow \
        sklearn \
        opencv-python \   
        pillow \
        minio
EOF
```

```sh
# docker build script
REGISTRY=kubeflow-registry.default.svc.cluster.local:30000
TAG=$REGISTRY/kf-base:latest

docker build -f Dockerfile.kf-base -t $TAG . && \
docker push $TAG
```

# Method 1. Host에서 command로 실행

### Step 1. docker 명령으로 build
**Host에 ssh 접속**하여 `01-covid19-katib-train.py`를 업로드한 후

사전준비에서 마련한 Base Image를 이용하여 아래와 같이 docker build하고 push한다

```sh
# docker build script
REGISTRY=kubeflow-registry.default.svc.cluster.local:30000
TAG=$REGISTRY/covid19-katib-job:latest

cat << EOF > Dockerfile.covid 
FROM $REGISTRY/kf-base:latest
COPY 01-covid19-katib-train.py /app/
CMD ['python3', '/app/01-covid19-katib-train.py']
EOF

docker build -f Dockerfile.covid -t $TAG . && \
docker push $TAG
```

### Setp 2. kubectl로 apply
Host에 ssh 접속하여 아래와 같이 kubectl apply 실행
```sh
kubectl apply -f 03-covid19-katib-random.yaml
```

# Method 2. JupyterNotebook에서 실행

### Step 1. fairing으로 build
아래 셀을 실행하여 fairing으로 image build하면 image tag명이난수로 생성된다
(기존 fairing에서 build만 실행하기 실습과 동일한 코드이다)

In [2]:
from kubeflow.fairing.preprocessors.base import BasePreProcessor 
from kubeflow.fairing.builders.append.append import AppendBuilder

preproc = BasePreProcessor(
    input_files = ['01-covid19-katib-train.py'], 
    #path_prefix = '/myapp', # default: /app
    command = ['python3'],  # default: python
)    

PRIVATE_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'

builder = AppendBuilder(
    preprocessor = preproc,
    # 사전준비에서 마련한 Base Image
    base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', 
    image_name = 'covid19-katib-job',
    registry = PRIVATE_REGISTRY,
    push = True,
)

builder.build()

print(f'Published Docker image with tag: "{builder.image_tag}"')

[W 201205 19:20:21 append:50] Building image using Append builder...
[I 201205 19:20:21 base:107] Creating docker context: /tmp/fairing_context_4bvlie4_
[I 201205 19:20:21 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.cluster.local:30000/kf-base:latest'
[W 201205 19:20:21 append:54] Image successfully built in 0.0235400310048135s.
[W 201205 19:20:21 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/covid19-katib-job:73C87772...
[I 201205 19:20:21 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.cluster.local:30000/covid19-katib-job:73C87772'
[W 201205 19:20:21 append:81] Uploading kubeflow-registry.default.svc.cluster.local:30000/covid19-katib-job:73C87772
[I 201205 19:20:21 docker_session_:280] Layer sha256:2a978df167d497637064cfdf8bd33ad6c84b4611a08090ba370bb73f1adb892e exists, skipping
[I 201205 19:20:21 docker_session_:280] Layer sha256:3cf8fb62ba5ffb221a2edb2208741346e

Published Docker image with tag: "kubeflow-registry.default.svc.cluster.local:30000/covid19-katib-job:73C87772"


### Step 2. katib UI에서 submit
katib UI의 submit 화면에서 `03-covid19-katib-random.yaml` 내용을 copy & paste하고 submit 버튼 클릭

Step 1.에서 fairing이 생성한 image tag명으로 `03-covid19-katib-random.yaml`을 수정한다
```yaml
apiVersion: "kubeflow.org/v1alpha3"
kind: Experiment
...
spec:
...
  trialTemplate:
    goTemplate:
...
          spec:
            template:
              spec:
                containers:
                - name: {{.Trial}}
                  image: kubeflow-registry.default.svc.cluster.local:30000/katib-job:latest  # 여기를 수정
```

# 관련 SDK API
- https://kubeflow-fairing.readthedocs.io/en/latest/source/kubeflow.fairing.preprocessors.html#module-kubeflow.fairing.preprocessors.base
- https://kubeflow-fairing.readthedocs.io/en/latest/source/kubeflow.fairing.builders.append.html#module-kubeflow.fairing.builders.append.append

# 참고: Notebook Preprocessor
```
from kubeflow.fairing.preprocessors.converted_notebook import ConvertNotebookPreprocessor
preproc = ConvertNotebookPreprocessor(
    notebook_file="example.ipynb"
)    
```